In [1]:
from wilcoxon.spiderman import *
import re
import ast
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm

In [19]:
def getHSKVocab(level):
    site = website('https://hsk.academy/en/hsk_%d' % level)
    soup = site.html('script',string=re.compile('JSON'))[0]
    words = re.findall('{"id".*?}',soup.string.replace('\\',''))
    words = [ast.literal_eval(x) for x in words]
    df = pd.DataFrame(words)[['hanziRaw','trad','pinyinToneSpace','def']]
    df.columns = ['Word','Traditional','Word Pinyin','Usages']
    df['Level'] = 'HSK %d' % level
    return df

def getSentence(word):
    try:
        site = website('https://hsk.academy/en/words/%s' % word)
        results = []
        for span in site.html('span'):
            if span.parent.name == 'li':
                results.append(list(span.stripped_strings))
        if results == []:
            return [['','']]
        return results
    except: return [['','']]
    
def fullProcess():
    hsk = pd.concat([getHSKVocab(level) for level in range(1,7)])
    results = []
    for word in tqdm(hsk['Word']):
        results.append(getSentence(word))
    hsk['results'] = results
    hsk = hsk.explode('results')
    hsk['Sentence'] = hsk['results'].apply(lambda x: x[0])
    hsk['Sentence Pinyin'] = hsk['results'].apply(lambda x: x[1])
    hsk = hsk.drop('results', axis=1)
    return hsk.reset_index(drop=True)

In [8]:
hsk = fullProcess()

100%|██████████| 5000/5000 [48:07<00:00,  1.73it/s] 


In [21]:
hsk[hsk['Meaning'] != ''].tail(60)

,Word,Traditional,Word Pinyin,Usages,Level,Sentence,Meaning
1058,信任,信任,xìn rèn,to trust; to have confidence in,HSK 5,由于他工作认真，因此得到了大家的信任。,"Yóuyú tā gōngzuò rènzhēn, yīncǐ dédàole dàjiā ..."
1062,形式,形式,xíng shì,shape; outer appearance; form; CL:个,HSK 5,内容是最主要的，其次才是形式。,"Nèiróng shì zuì zhǔyào de, qícì cái shì xíngshì."
1102,夜,夜,yè,night; evening,HSK 5,我昨天夜里 3 点就醒了，然后就再也睡不着,"Wǒ zuótiān yèlǐ 3 diǎn jiù xǐngle, ránhòu jiù ..."
1113,乙,乙,yǐ,second in order; second of 10 heavenly stems 十...,HSK 5,甲：谢谢你！乙：不客气。,Jiǎ: Xièxiè nǐ! Yǐ: Bù kèqì.
1113,乙,乙,yǐ,second in order; second of 10 heavenly stems 十...,HSK 5,甲：对不起。乙：没关系。,Jiǎ: Duìbùqǐ. Yǐ: Méiguānxì.
1113,乙,乙,yǐ,second in order; second of 10 heavenly stems 十...,HSK 5,甲：对不起！乙：没关系。,Jiǎ: Duìbùqǐ! Yǐ: Méiguānxì.
1113,乙,乙,yǐ,second in order; second of 10 heavenly stems 十...,HSK 5,乙：是，我是中国人。,"Yǐ: Shì, wǒ shì zhōngguó rén."
1113,乙,乙,yǐ,second in order; second of 10 heavenly stems 十...,HSK 5,甲：谢谢你。乙：不客气！,Jiǎ: Xièxiè nǐ. Yǐ: Bù kèqì!
1123,银,銀,yín,silver; silver-colored; relating to money or c...,HSK 5,我去银行换点儿钱。,Wǒ qù yínháng huàn diǎn er qián.
1123,银,銀,yín,silver; silver-colored; relating to money or c...,HSK 5,我想去银行办一张信用卡。,Wǒ xiǎng qù yínháng bàn yī zhāng xìnyòngkǎ.
